# Orientation Estimation


## Reference

- https://towardsdatascience.com/matplotlib-animations-in-jupyter-notebook-4422e4f0e389
    
- https://matplotlib.org/stable/gallery/animation/simple_anim.html 

- https://seanng.my/2020/07/15/wireless-imu-sensor-using-bluetooth-low-energy-ble/

- https://roboticsclubiitk.github.io/2017/12/21/Beginners-Guide-to-IMU.html

- Complementary filter https://www.pieter-jan.com/node/11

- Arduino IMU https://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_accelerometer

- https://stackoverflow.com/questions/23009549/roll-pitch-yaw-calculation

- https://habr.com/en/post/499190/

 - https://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_magnetometer
 
 - https://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_gyroscope
 
- https://www.infineon.com/dgdl/Infineon-3D_Magnetic_Sensor_for_Angle_Measurements-AN-v01_10-AN-v01_01-EN.pdf?fileId=5546d46265f064ff01665a3d22f055e3

![style=centerme](....png)

In [1]:
import matplotlib.pylab as plt
import numpy as np
import matplotlib.animation as animation
import pandas as pd
import time 
import collections
import serial
import threading

In [12]:
%pylab

Using matplotlib backend: MacOSX
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 287, in process
    func(*args, **kwargs)
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/matplotlib/animation.py", line 913, in _start
    self.event_source.add_callback(self._step)
AttributeError: 'NoneType' object has no attribute 'add_callback'


In [13]:
import numpy as np
#collections.deque(np.arange(1000), maxlen=1000),

In [14]:
class arduino(object):#serial_port,baud_rate=baud_rate):
    
    def __init__(self,serial_port='/dev/cu.usbmodem14201',baud_rate=9600,vector_lenght=1000,alpha=0.02):
        
        self.serial_port   = serial_port
        self.baud_rate     = baud_rate
        
        self.alpha         = 0.02
        
        self.vector_lenght = vector_lenght
        
        self.Timestamp    = collections.deque(np.arange(vector_lenght), maxlen=vector_lenght)
        self.TimeInterval = collections.deque(np.arange(vector_lenght), maxlen=vector_lenght)
        
        self.Accelerometer = {'x':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'y':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'z':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}
        self.Gyroscope     = {'x':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'y':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'z':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}
        self.Magnetometer  = {'x':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'y':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'z':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}
        
       
        self.Roll   = {'Accelerometer':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Gyroscope'    :collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Magnetometer' :collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Complementary':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}

    def read_serial_monitor(self):
    
        arduinoString = self.arduinoData.readline()
        arduinoString = arduinoString.decode("utf-8")
        arduinoString = arduinoString.strip()

        self.char_list     = arduinoString.split('\t')  
        
        #print("\r{:}".format(self.char_list),end='')
        #print("\rtest",end='')
        
        
    def wait_serial_monitor(self):
        
        self.read_serial_monitor()
        while len(self.char_list)!=10: 
            print("\rConnecting...",end='')
            self.read_serial_monitor()
        print("\rConnected     ",end='')


    def get_new_data(self):
    
        self.read_serial_monitor()
        
        timestamp,ax,ay,az,gx,gy,gz,mx,my,mz = self.char_list
        
        if mz == 'ovf': 
            mz = 0
            
        timestamp,ax,ay,az,gx,gy,gz,mx,my,mz = float(timestamp),float(ax),float(ay),float(az),float(gx),float(gy),float(gz),float(mx),float(my),float(mz)

        timestamp /= 1000
        gx,gy,gz  = np.deg2rad(gx),np.deg2rad(gy),np.deg2rad(gz)
        
        self.update_raw_vectors(timestamp,ax,ay,az,gx,gy,gz,mx,my,mz)
        
    def update_raw_vectors(self,timestamp,ax,ay,az,gx,gy,gz,mx,my,mz):
        
        self.Timestamp.append(float(timestamp))
        self.TimeInterval.append(self.Timestamp[-1]-self.Timestamp[-2])
        self.Accelerometer['x'].append(ax);self.Accelerometer['y'].append(ay);self.Accelerometer['z'].append(az)
        self.Gyroscope['x'].append(gx);    self.Gyroscope['y'].append(gy);    self.Gyroscope['z'].append(gz)
        self.Magnetometer['x'].append(mx); self.Magnetometer['y'].append(my); self.Magnetometer['z'].append(mz)

   
    def read(self):
            
        self.arduinoData = serial.Serial(self.serial_port, self.baud_rate) 
        
        self.wait_serial_monitor()
        self.get_new_data()
        
        while True:
            
            self.get_new_data()
            
            alpha = 0.02
            
            self.Roll['Accelerometer'].append( np.arctan2(self.Accelerometer['y'][-1],self.Accelerometer['x'][-1]))
            self.Roll['Gyroscope'].append(self.Gyroscope['z'][-1]*self.TimeInterval[-1]+self.Roll['Gyroscope'][-1])
            #self.Roll['Magnetometer']
            self.Roll['Complementary'].append((1-alpha)*(self.Roll['Complementary'][-1]  + self.Gyroscope['z'][-1]*self.TimeInterval[-1]) + alpha*self.Roll['Accelerometer'][-1])
            
            if (self.Magnetometer['x'][-1]==0)&(self.Magnetometer['y'][-1]==0)&(self.Magnetometer['z'][-1]==0):
                
                self.Magnetometer['x'][-1],self.Magnetometer['y'][-1],self.Magnetometer['z'][-1]=self.Magnetometer['x'][-2],self.Magnetometer['y'][-2],self.Magnetometer['z'][-2]

            
            

# Yaw Pitch Roll

In [17]:
my_arduino = arduino()
#Loops the in_background function
try:
    thread = threading.Thread(target = my_arduino.read)
    thread.start() 

except KeyboardInterrupt:
    thread._Thread_stop() 
    #sys.exit()
    
fig, ax3 = plt.subplots(1,1,figsize=[6.4,4.8])

#RollAccelerometer, = ax3.plot(my_arduino.Roll['Accelerometer'],np.arange(1000))
#RollGyroscope,     = ax3.plot(my_arduino.Roll['Gyroscope'],np.arange(1000))
RollComplementary, = ax3.plot(my_arduino.Roll['Complementary'],np.arange(1000))



#lim = [-np.pi*1.1,np.pi*1.1]
#ax1.set_ylim(lim);ax2.set_ylim(lim);ax3.set_ylim(lim)

ax3.set_xticks([-np.pi,-np.deg2rad(16),-np.deg2rad(5),0,np.deg2rad(5),np.deg2rad(16),np.pi],["-90","-16","-5","0","5","16","90"])
ax3.set_xlim([-np.deg2rad(20),np.deg2rad(20)])
ax3.grid(True)

def animate(i):
    
    
    #RollAccelerometer.set_xdata(my_arduino.Roll['Accelerometer'])
    #RollGyroscope.set_xdata(my_arduino.Roll['Gyroscope'])
    RollComplementary.set_xdata(my_arduino.Roll['Complementary'])

ani = animation.FuncAnimation(fig, animate, interval=200)

Connected     

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/serial/serialposix.py", line 575, in read
    buf = os.read(self.fd, size - len(read))
OSError: [Errno 6] Device not configured

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_53981/2852808428.py", line 86, in read
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_53981/2852808428.py", line 54, in get_new_data
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_53981/2852808428.py", line 33, in read_serial_monitor
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/se